In [1]:
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bionlp13cg_md-0.5.1.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 19.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.6.1-py2.py3-none-any.whl.metadata (8.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 51.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 42.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 77.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.8/204.8 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 28.0 MB

In [4]:
import pandas as pd
import numpy as np
import spacy
from transformers import pipeline
from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = pd.read_csv("Synthetic_HCUP-Structured_Data.csv")

In [6]:
admission_type_mapping = {
    1: "Emergency", 2: "Urgent", 3: "Elective", 5: "Trauma", 9: "Unknown"
}
discharge_status_mapping = {
    1: "Home/self-care", 2: "Transferred to hospital", 3: "Transferred to SNF",
    6: "Home health care", 7: "Left against medical advice", 30: "Still patient",
    50: "Hospice (home)", 51: "Hospice (facility)", 20: "Expired",
    43: "Transferred to federal hospital", 62: "Transferred to rehab", 63: "Transferred to long-term care"
}

df['Admission_Type_Text'] = df['ADMSN_TYPE'].map(admission_type_mapping).fillna('Unknown')
df['Discharge_Status_Text'] = df['DSCHRG_STUS'].map(discharge_status_mapping).fillna('Other')

In [7]:
generator = pipeline("text2text-generation", model="google/flan-t5-base", max_length=256)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cuda:0


In [8]:
def create_prompt(row):
    return f"""
Patient admission type: {row['Admission_Type_Text']}.
Length of stay: {row['LOS']} days.
Discharge status: {row['Discharge_Status_Text']}.
Attending specialty: {row['AT_SPCLTY']}.
Plan paid amount: ${row['PLAN_PMT_AMT']}.
Generate a detailed clinical admission and discharge note.
"""

tqdm.pandas()
df["Prompt"] = df.progress_apply(create_prompt, axis=1)
df["Generated_Note"] = df["Prompt"].progress_apply(lambda x: generator(x)[0]["generated_text"])

100%|██████████| 10000/10000 [1:28:43<00:00,  1.88it/s] 


In [9]:
nlp = spacy.load("en_ner_bionlp13cg_md")

def extract_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

df["Extracted_Entities"] = df["Generated_Note"].progress_apply(extract_entities)

100%|██████████| 10000/10000 [00:52<00:00, 192.12it/s]


In [10]:
output_cols = [
    "PERSON_ID", "Admission_Type_Text", "Discharge_Status_Text",
    "LOS", "AT_SPCLTY", "PLAN_PMT_AMT", "Generated_Note", "Extracted_Entities"
]
df[output_cols].to_csv("hcup_notes_with_entities.csv", index=False)
print(" File saved: hcup_notes_with_entities.csv")

 File saved: hcup_notes_with_entities.csv


In [11]:
!pip install datasets transformers scikit-learn seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached scikit_learn-1.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached scipy-1.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.5 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.6 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=346452ddab10544f4ce4b80107c9d4a48bd3557cd0c059cfd6b3f196fbfec018
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abf

In [83]:
pip install transformers datasets accelerate peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 41.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [84]:
from datasets import load_dataset
# Load MedQA dataset
dataset = load_dataset("GBaker/MedQA-USMLE-4-options")["train"]

In [123]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
import torch

# Step 1: Load model and tokenizer (no fp16, no quantization)
model_id = "stanford-crfm/biomedlm"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token  # Fix padding error

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float32  
)

# Step 2: Apply LoRA
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["c_attn"], 
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

/opt/conda/lib/python3.10/site-packages/accelerate/utils/modeling.py:1569: UserWarning: Current model requires 4194432.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


trainable params: 5,242,880 || all params: 2,599,490,560 || trainable%: 0.2017


In [127]:
def format_medqa(example):
    question = example["question"]
    options = example["options"]
    answer = ord(example["answer_idx"].strip().upper()) - 65 
    
    prompt = f"Q: {question}\nOptions:\n"
    for i, opt in enumerate(options):
        prompt += f"{chr(65 + i)}. {opt}\n"
    
    prompt += "A:"
    response = f" {chr(65 + answer)}"
    
    return {"text": prompt + response}

# Re-map the dataset with corrected function
formatted_data = dataset.map(format_medqa)

Map: 100%|██████████| 10178/10178 [00:00<00:00, 12870.84 examples/s]


In [128]:
from transformers import DataCollatorForLanguageModeling

def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_dataset = formatted_data.map(tokenize, batched=True)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Map: 100%|██████████| 10178/10178 [00:03<00:00, 2890.33 examples/s]


In [131]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./biomedlm-medqa-lora",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    max_steps=300,                    
    logging_steps=10,
    save_strategy="no",            
    learning_rate=5e-5,
    report_to="none",
    fp16=False                        
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset, 
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

/tmp/ipykernel_1913/1005975040.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,3.486600
20,3.460500
30,3.403900
40,3.416100
50,3.334600
60,3.252600
70,3.216500
80,3.200500
90,3.078600
100,3.042000


TrainOutput(global_step=300, training_loss=2.982263552347819, metrics={'train_runtime': 8196.6341, 'train_samples_per_second': 0.293, 'train_steps_per_second': 0.037, 'total_flos': 1.8600802910208e+16, 'train_loss': 2.982263552347819, 'epoch': 0.2356637863315004})

In [132]:
# Save final model
model.save_pretrained("./biomedlm-medqa-lora")
tokenizer.save_pretrained("./biomedlm-medqa-lora")

('./biomedlm-medqa-lora/tokenizer_config.json',
 './biomedlm-medqa-lora/special_tokens_map.json',
 './biomedlm-medqa-lora/vocab.json',
 './biomedlm-medqa-lora/merges.txt',
 './biomedlm-medqa-lora/added_tokens.json',
 './biomedlm-medqa-lora/tokenizer.json')

In [22]:
!pip install sentence-transformers faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 51.2 MB/s eta 0:00:0000:0100:01


In [14]:
import pandas as pd
import numpy as np
import faiss
import pickle
from sentence_transformers import SentenceTransformer

# Load HCUP notes
df_hcup = pd.read_csv("hcup_notes_with_entities.csv")

# Step 1: Embed notes
embedder = SentenceTransformer("all-MiniLM-L6-v2")
corpus = df_hcup["Generated_Note"].tolist()
embeddings = embedder.encode(corpus, show_progress_bar=True, convert_to_numpy=True)

# Step 2: Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Step 3: Save index and metadata
faiss.write_index(index, "hcup_faiss_index.idx")

with open("hcup_id_mapping.pkl", "wb") as f:
    pickle.dump(df_hcup[["PERSON_ID", "Generated_Note"]].to_dict(orient="records"), f)

print(" FAISS index and ID mapping saved.")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Batches: 100%|██████████| 313/313 [00:02<00:00, 128.27it/s]


 FAISS index and ID mapping saved.


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from sentence_transformers import SentenceTransformer
import faiss
import pickle
import re
import pandas as pd

In [6]:
# Load tokenizer and base model
base_model_id = "stanford-crfm/biomedlm"
adapter_path = "/kaggle/input/gen_ai_proj/pytorch/default/1/biomedlm-diagnosis-finetuned"

tokenizer = AutoTokenizer.from_pretrained(base_model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(base_model_id)
model = PeftModel.from_pretrained(model, adapter_path)

tokenizer_config.json:   0%|          | 0.00/267 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/602k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/276k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/876 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['corda_config', 'trainable_token_indices'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


In [7]:
# Move model to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [9]:
# Load FAISS index and ID mapping
index = faiss.read_index("/kaggle/input/gen_ai_proj/pytorch/default/1/hcup_faiss_index.idx")
with open("/kaggle/input/gen_ai_proj/pytorch/default/1/hcup_id_mapping.pkl", "rb") as f:
    id_mapping = pickle.load(f)

In [10]:
# Load sentence transformer for embeddings
embedder = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
# Load additional CSV-based context
csv_df = pd.read_csv("/kaggle/input/synthetic-diagnosis-qa-dataset/Synthetic_Diagnosis_QA_Dataset.csv")
csv_contexts = csv_df["prompt"].tolist()

In [13]:
# Utility: Retrieve top-k FAISS notes
def retrieve_similar_notes(query, k=3):
    query_vec = embedder.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_vec, k)
    return [id_mapping[i]["Generated_Note"] for i in indices[0]]

In [14]:
# Utility: Retrieve top-k similar prompts from CSV
def retrieve_context_from_csv(query, top_k=1):
    csv_vectors = embedder.encode(csv_contexts, convert_to_tensor=True)
    query_vec = embedder.encode(query, convert_to_tensor=True)
    scores = torch.nn.functional.cosine_similarity(query_vec, csv_vectors)
    top_idx = torch.topk(scores, k=top_k).indices
    return [csv_contexts[i] for i in top_idx]

In [15]:
# Build prompt

def build_diagnosis_prompt(patient_info, historical_notes, csv_context=None):
    prompt = (
        "You are a clinical diagnosis assistant.\n\n"
        "Your task is to analyze the patient's symptoms and history and provide the most likely diagnosis with a concise medical explanation.\n\n"
        f"### Patient Summary:\n{patient_info.strip()}\n\n"
        f"### Historical Notes:\n" + "\n".join(f"- {note.strip()}" for note in historical_notes)
    )

    if csv_context:
        prompt += (
            f"\n\n### Additional Clinical Context:\n"
            + "\n".join(f"- {ctx.strip()}" for ctx in csv_context)
        )

    prompt += (
        "\n\n### Task:\n"
        "What is the most likely diagnosis and why?\n\n"
        "Diagnosis and Explanation:"
    )
    return prompt

In [16]:
# Clean hallucinated artifacts from response
import re
import nltk
nltk.download("punkt")
from nltk.tokenize import sent_tokenize

def clean_response(text):
    import re

    # Remove markdown images and formatting artifacts
    text = re.sub(r"!\[.*?\]\(.*?\)", "", text)
    text = re.sub(r">+", "", text)
    text = re.sub(r"\d+\.", "", text)
    text = re.sub(r"[•\-:]+", " ", text)

    # Normalize specific known noisy phrases
    text = re.sub(r"\b(Hypertension)\s*[:]?[\s]*\b", "Hypertension. ", text, flags=re.IGNORECASE)
    text = re.sub(r"\b(Acute coronary syndrome)\s*[:]?[\s]*\b", "Acute coronary syndrome. ", text, flags=re.IGNORECASE)
    text = re.sub(r"\b(Plan|Answer|Question|Diagnosis)\s*[:]?[\s]*", "", text, flags=re.IGNORECASE)

    # Remove questions and incomplete phrases
    lines = re.split(r"[.\n]", text)
    filtered = []
    for line in lines:
        line = line.strip()
        if not line:
            continue
        if line.endswith("?"):
            continue
        if re.match(r"(?i)^(what|why|how|when|is|are|can|do|does|did|your|who|where|which|¿)\b", line):
            continue
        filtered.append(line)

    # Combine back to paragraph
    clean_text = ". ".join(filtered).strip()

    # Final fix: punctuation and whitespace
    clean_text = re.sub(r"\s{2,}", " ", clean_text)
    if clean_text and not clean_text.endswith("."):
        clean_text += "."

    return clean_text

# Main inference function
def suggest_diagnosis(patient_info, k=3, csv_top_k=1, max_tokens=200):
    faiss_notes = retrieve_similar_notes(patient_info, k=k)
    csv_context = retrieve_context_from_csv(patient_info, top_k=csv_top_k)

    prompt = build_diagnosis_prompt(patient_info, faiss_notes, csv_context=csv_context)

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=512).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            num_beams=4,
            do_sample=False,
            early_stopping=True,
            pad_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=3,
            repetition_penalty=1.2
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Cut off any prefix text
    if "Diagnosis and Explanation:" in decoded:
        decoded = decoded.split("Diagnosis and Explanation:")[-1]

    return clean_response(decoded)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
sample_structured_queries = [
    "Patient admission type: Emergency. Length of stay: 3 days. Discharge status: Home/self-care. Attending specialty: Gastroenterology. Plan paid amount: $2200.",
    "Patient admission type: Elective. Length of stay: 4 days. Discharge status: Home/self-care. Attending specialty: Gastroenterology. Plan paid amount: $2500.",
    "Patient admission type: Urgent. Length of stay: 2 days. Discharge status: Skilled Nursing Facility. Attending specialty: Gastroenterology. Plan paid amount: $1800.",
    "Patient admission type: Trauma. Length of stay: 1 day. Discharge status: Transferred to another hospital. Attending specialty: Gastroenterology. Plan paid amount: $1900.",
    "Patient admission type: Emergency. Length of stay: 5 days. Discharge status: Deceased. Attending specialty: Gastroenterology. Plan paid amount: $3000.",
    "Patient admission type: Emergency. Length of stay: 6 days. Discharge status: Home/self-care. Attending specialty: Gastroenterology. Plan paid amount: $2700.",
    "Patient admission type: Elective. Length of stay: 2 days. Discharge status: Home/self-care. Attending specialty: Gastroenterology. Plan paid amount: $1700.",
    "Patient admission type: Emergency. Length of stay: 3 days. Discharge status: Discharged to rehab. Attending specialty: Gastroenterology. Plan paid amount: $2100.",
    "Patient admission type: Urgent. Length of stay: 3 days. Discharge status: Home/self-care. Attending specialty: Gastroenterology. Plan paid amount: $2000.",
    "Patient admission type: Emergency. Length of stay: 2 days. Discharge status: Home/self-care. Attending specialty: Gastroenterology. Plan paid amount: $1600."
]

In [20]:
for query in sample_queries:
    print("Prompt:\n", query)
    print("Diagnosis and Explanation:\n", suggest_diagnosis(query))
    print("="*80)

Prompt:
 Patient admission type: Emergency.
Length of stay: 3 days.
Discharge status: Home/self-care.
Attending specialty: Gastroenterology.
Plan payment amount: $2200.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Diagnosis and Explanation:
 Table Place the patient in the emergency room. The patient was admitted to the emergency department with a chief complaint of abdominal pain. The patient was discharged from the hospital. In this case, the patient was referred to the gastroenterology consultant. You are an attending physician. Discuss your Place your patient. Diagnose and treat your patient? and Treatment Clinical. and treatment. ned. Admit him/her? Specialty paid for inpatient? ned primary care physician. The gastroenterologist? Consult? Discharge? Or should he/she? Primary care. The purpose of this study was to determine whether you are an inpatient or outpatient? {#Sec5}. ===gastroenterology?}.
Prompt:
 Patient admission type: Urgent.
Length of stay: 2 days.
Discharge status: Skilled Nursing Facility.
Attending specialty: Gastroenterology.
Plan payment amount: $1800.


...

Diagnosis and Explanation:
 {#Sec8}. The patient was admitted to the emergency department (ED). Explanatory notes. Payment type The patient was diagnosed with an acute abdomen Acute cholecystitis, pancreatitis, acute appendicitis. She was diagnosed as a case of acute cholecystitis. Appendicitis is a of acute abdominal pain in the setting of pregnancy. You are an obstetrician/gastroenterologist, and you're an emergency medicine physician A pregnant patient with a right upper quadrant abdominal pain during pregnancy. The patient's history 30 day admission status 1 day. $$\documentclass[12pt]{minimal}. \usepackage{amsmath}. $$\end{document}$.
Prompt:
 Patient admission type: Emergency.
Length of stay: 2 days.
Discharge status: Home/self-care.
Attending specialty: Gastroenterology.
Plan payment amount: $1600.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Diagnosis and Explanation:
 The patient was admitted to the hospital on day. The patient was referred to the emergency department (ED). A of acute pancreatitis was made on day 3, and you are a gastroenterology consultant. Admission Acute pancreatitis was diagnosed based on history and physical exam findings consistent with acute pancreatitis. Initial presentation You are an inpatient. The patient presented with epigastric pain to the ED. Admitted Clinical Abdominal Pain. Abdominal pain what is your ? Place of origin Emergency medicine. Pancreatic cancer. Admission. Diagnose the cause of the abdominal pain A of pancreatic cancer. Diagnose and treat the patient. Acute pancreatitis Clinical context. Chronic pancreatitis. acute pancreatitis Pancreas. The of chronic pancreatitis.


In [22]:
# Sample test
query = query = "Patient presents with shortness of breath, chest tightness, and was admitted for 3 days under cardiology."
diagnosis = suggest_diagnosis(query)
print("Suggested Diagnosis and Explanation:\n", diagnosis)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Suggested Diagnosis and Explanation:
 The patient's primary complaint. A patient with a chief complaint of chest pain presented to the emergency department (ED) with chest pain. Provides a focused history and physical exam findings. Chest pain,. ned. Case 2 To review the clinical context. This case was presented at the end of the day case patient was diagnosed with acute chest pain is admitted to your patient summary. In this case, the patient was discharged from the hospital with a short stay patient was referred to the ED. Due to the high prevalence of pulmonary embolism (PE) Pulmonary embolism is a potentially life threatening complication of deep venous thrombosis (DVT) is one of the most common cause of deep vein thrombosis. Due to the rarity of the DVT. of PE is challenging.


In [53]:
import evaluate
import pandas as pd

# Load and sample test CSV
test_df = pd.read_csv("Synthetic_Diagnosis_Prompts.csv")
test_df = test_df.sample(n=25, random_state=42)

# Load metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

# Evaluation function
def compute_nlg_metrics(predictions, references):
    bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
    rouge_score = rouge.compute(predictions=predictions, references=references)
    meteor_score = meteor.compute(predictions=predictions, references=references)

    return {
        "BLEU": bleu_score['bleu'],
        "ROUGE-L": rouge_score['rougeL'],
        "METEOR": meteor_score['meteor']
    }

# Optional: keyword coverage metric
clinical_terms = ["hypertension", "acute coronary syndrome", "myocardial infarction", "chest pain", "troponin", "ecg", "long-term care"]

def keyword_coverage(prediction):
    found = [term for term in clinical_terms if term in prediction.lower()]
    return len(found) / len(clinical_terms)

# Run predictions
preds = []
refs = []

for _, row in test_df.iterrows():
    pred = suggest_diagnosis(row['prompt'])
    preds.append(pred)
    refs.append(row['diagnosis'])

# Compute metrics
results = compute_nlg_metrics(preds, refs)
print(results)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


{'BLEU': 0.0, 'ROUGE-L': 0.07842936873243765, 'METEOR': 0.1229333670009592}
